In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_percentage_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import random
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer, PowerTransformer, RobustScaler

In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold,StratifiedKFold

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer, PowerTransformer, RobustScaler
from sklearn.decomposition import PCA

from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor,AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
import xgboost
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

from keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import (mean_squared_error,
                             r2_score,
                             mean_absolute_percentage_error,
                             mean_absolute_error
                            )

from sklearn.pipeline import Pipeline
from statsmodels.tsa.api import VAR

from sklearn.feature_selection import SelectKBest,f_regression

In [3]:
df = pd.read_csv('../../data/dataset/Full_data_File.csv')

In [4]:
df['Date'] = pd.to_datetime(df['Date'])

In [5]:
df.shape

(49674, 27)

In [7]:
df['location'].unique()

array(['Amsterdam', 'Athens', 'Barcelona', 'Berlin', 'Bruges', 'Brussels',
       'Budapest', 'Cluj-Napoca', 'Copenhagen', 'Frankfurt Airport',
       'Ghent', 'Graz Airport', 'Helsinki', 'Isle of Man',
       'Istanbul Airport', 'Limerick', 'London', 'Lyon', 'Madrid',
       'Marseille', 'Milan', 'Moscow', 'Munich', 'Nice', 'Nicosia',
       'Oslo', 'Paris', 'Poznan', 'Prague', 'Reykjavik Airport', 'Rome',
       'Stockholm', 'bern', 'luxembourg findel airport'], dtype=object)

In [11]:
# df = df[df['location'].isin(['Marseille', 'Milan', 'Moscow', 'Munich', 'Nice', 'Nicosia',
#        'Oslo', 'Paris', 'Poznan', 'Prague', 'Reykjavik Airport', 'Rome',
#        'Stockholm', 'bern', 'luxembourg findel airport'])].copy()

In [12]:
import tensorflow as tf
def NN_Regression(X_train, X_test, y_train, y_test, neurons_layer1, neurons_layer2, learning_rate, batch_size):
    X_train_scaled = X_train
    X_test_scaled = X_test
    

    random.seed(1)
    np.random.seed(1)
    tf.keras.utils.set_random_seed(1)

    # Build the Neural Network model
    def build_nn(input_dim):
        model = Sequential([
            Input(shape=(input_dim,)),  # Define input shape
            Dense(neurons_layer1, activation='relu'),
            Dense(neurons_layer2, activation='relu'),
            Dense(1, activation='linear')  # Regression output layer
        ])
        model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')  # Compile the model
        return model

    nn = build_nn(X_train_scaled.shape[1])

    # Train the Neural Network
    nn.fit(X_train_scaled, y_train, epochs=50, batch_size=batch_size, validation_split=0.2, verbose=0)

    # Predict using the Neural Network
    y_pred = nn.predict(X_test_scaled).flatten()

    # Evaluate the Neural Network
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r_2 = r2_score(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100

    return mae, rmse, r_2, mape, y_pred


In [13]:
def fitness_function(params, X_train, X_test, y_train, y_test):
    # Decode parameters
    neurons_layer1 = int(params[0])
    neurons_layer2 = int(params[1])
    learning_rate = params[2]
    batch_size = int(params[3])
    scaler_name = 'not_processing'  # Fixed scaler for simplicity

    # Run NN Regression and get all metrics
    mae, rmse, r_2, mape, y_pred = NN_Regression(
        X_train, X_test, y_train, y_test,
        neurons_layer1, neurons_layer2, learning_rate, batch_size
    )
    return mae, rmse, r_2, mape, y_pred


In [14]:
def pso_nn_optimization(X_train, X_test, y_train, y_test, n_particles=10, n_iterations=20):
    # Define parameter ranges
    param_ranges = [
        (10, 100),  # neurons_layer1
        (10, 100),  # neurons_layer2
        (0.0001, 0.01),  # learning_rate
        (16, 128)  # batch_size
    ]

    # Initialize particles
    particles = [
        [random.uniform(low, high) for low, high in param_ranges]
        for _ in range(n_particles)
    ]
    velocities = [np.random.uniform(-1, 1, len(param_ranges)) for _ in range(n_particles)]
    personal_best_positions = particles.copy()
    personal_best_scores = [float('inf')] * n_particles
    global_best_position = None
    global_best_metrics = {'mae': float('inf'), 'rmse': None, 'r2': None, 'mape': None, 'y_pred': None}

    # PSO loop
    for iteration in range(n_iterations):
        for i, particle in enumerate(particles):
            # Evaluate fitness
            mae, rmse, r_2, mape, y_pred = fitness_function(particle, X_train, X_test, y_train, y_test)
            
            # Update personal bests
            if mae < personal_best_scores[i]:
                personal_best_scores[i] = mae
                personal_best_positions[i] = particle.copy()
            
            # Update global best
            if mae < global_best_metrics['mae']:
                global_best_metrics = {
                    'mae': mae,
                    'rmse': rmse,
                    'r2': r_2,
                    'mape': mape,
                    'y_pred': y_pred
                }
                global_best_position = particle.copy()
        
        # Update particle velocities and positions
        for i, particle in enumerate(particles):
            r1, r2 = random.random(), random.random()
            cognitive = 1.5 * r1 * (np.array(personal_best_positions[i]) - np.array(particle))
            social = 1.5 * r2 * (np.array(global_best_position) - np.array(particle))
            velocities[i] = 0.5 * np.array(velocities[i]) + cognitive + social
            particles[i] = np.array(particle) + np.array(velocities[i])
            
            # Clip particles within parameter ranges
            particles[i] = np.clip(particles[i], [low for low, _ in param_ranges], [high for _, high in param_ranges])

        print(f"Iteration {iteration+1}/{n_iterations}, Best MAE so far: {global_best_metrics['mae']}")

    return global_best_position, global_best_metrics


In [15]:
def process_metadata(country_, city_, best_params, best_metrics):
    # Extract the best parameters from PSO results
    optimal_neurons_layer1 = int(best_params[0])
    optimal_neurons_layer2 = int(best_params[1])
    optimal_learning_rate = best_params[2]
    optimal_batch_size = int(best_params[3])
    optimal_mae = best_metrics['mae']
    optimal_rmse = best_metrics['rmse']
    optimal_r2 = best_metrics['r2']
    optimal_mape = best_metrics['mape']
    
    meta_data_list = []
    # add metadata to list
    meta_data_list.append(country_)
    meta_data_list.append(city_)
    meta_data_list.append(optimal_neurons_layer1)
    meta_data_list.append(optimal_neurons_layer2)
    meta_data_list.append(optimal_learning_rate)
    meta_data_list.append(optimal_batch_size)
    meta_data_list.append(optimal_mae)
    meta_data_list.append(optimal_rmse)
    meta_data_list.append(optimal_r2)
    meta_data_list.append(optimal_mape)
    
    meatdata_df = pd.DataFrame(meta_data_list).T
    meatdata_df.columns = [
     'country',
     'city',
     'optimal_neurons_layer1',
     'optimal_neurons_layer2',
     'optimal_learning_rate',
     'optimal_batch_size',
     'optimal_mae','optimal_rmse',
     'optimal_r2','optimal_mape'
    ]
    meatdata_df.to_csv('../_output_PSO+LSTM/metadata/summary_result_' + country_ + '_' + city_ + '.csv',index=False)
    
    full_output = pd.DataFrame({
        'actual_value' : y_test,
        'predict_value' : best_metrics['y_pred']
    })
    full_output['city'] = city_
    full_output['country'] = country_
    full_output.to_csv('../_output_PSO+LSTM/metadata/details_result_' + country_ + '_' + city_ + '.csv',index=False)

In [16]:
def get_k_fold_operation_data(df, date_, cv):
    
    df = pd.DataFrame(df)
    colLen = len(list(df))

    X = df.values[:, 0:colLen - 1]
    y = df.values[:, colLen - 1]
    
    X = MinMaxScaler().fit_transform(X)
    
    
    X_train_list = []
    X_test_list = []
    y_train_list = []
    y_test_list = []
    date_train_list = []
    date_test_list = []
    
    kf = KFold(n_splits=cv)
    
    for train_index,test_index in kf.split(X,y):
        X_train, X_test, y_train, y_test, date_train, date_test  = X[train_index], X[test_index], y[train_index], y[test_index], date_[train_index], date_[test_index]
        
        X_train_list.append(X_train)
        X_test_list.append(X_test)
        y_train_list.append(y_train)
        y_test_list.append(y_test)
        
        date_train_list.append(date_train)
        date_test_list.append(date_test)
        
    
        
    return X_train_list, X_test_list, y_train_list, y_test_list, date_train_list, date_test_list

In [17]:
def run_model_individually(optimal_neurons_layer1, 
                           optimal_neurons_layer2, 
                           optimal_learning_rate, 
                           optimal_batch_size,
                           X_train, 
                           X_test, 
                           y_train, 
                           y_test):

    # Set random seeds for consistency
    random.seed(1)
    np.random.seed(1)
    tf.keras.utils.set_random_seed(1)

    # Build the Neural Network model
    def build_nn(input_dim):
        model = Sequential([
            Input(shape=(input_dim,)),  # Define input shape
            Dense(optimal_neurons_layer1, activation='relu'),
            Dense(optimal_neurons_layer2, activation='relu'),
            Dense(1, activation='linear')  # Regression output layer
        ])
        model.compile(optimizer=Adam(learning_rate=optimal_learning_rate), loss='mse')  # Compile the model
        return model

    nn = build_nn(X_train.shape[1])

    # Train the Neural Network
    nn.fit(X_train, y_train, epochs=100, batch_size=optimal_batch_size, validation_split=0.2, verbose=0)
    
    # Predict on the test set
    y_pred = nn.predict(X_test).flatten()

    # Evaluate the model
    final_mae = mean_absolute_error(y_test, y_pred)
    final_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    final_r2 = r2_score(y_test, y_pred)
    final_mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    
    return final_mae, final_rmse, final_r2, final_mape, y_pred

In [18]:
cv = 5

for city_ in df['location'].unique():
    df_city = df[df['location'] == city_]
    country_ = df_city['country'].unique()[0]
    print(country_,city_)
    date_ = df_city['Date'].values
    
    df_city.drop(columns=['location', 'country', 'latitude', 'longitude', 'generationtime_ms','elevation','Date'],inplace=True)

    X = df_city.iloc[:,0:-1]
    y = df_city.iloc[:,-1]
    
    X = MinMaxScaler().fit_transform(X)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Run PSO to optimize NN hyperparameters
    best_params, best_metrics = pso_nn_optimization(X_train, X_test, y_train, y_test,n_particles=10, n_iterations=20)
    
    # Assign best parameter to variables
    optimal_neurons_layer1 = int(best_params[0])
    optimal_neurons_layer2 = int(best_params[1])
    optimal_learning_rate = best_params[2]
    optimal_batch_size = int(best_params[3])
    
    # process MetaData
    process_metadata(country_, city_, best_params, best_metrics)
    
    X_train_list, \
    X_test_list, \
    y_train_list, \
    y_test_list, \
    date_train_list, \
    date_test_list = get_k_fold_operation_data(df_city, date_, cv)
    
    k_fold = 1
    summary_value = []
    detail_value = []
    
    for X_train, X_test, y_train, y_test, date_train, date_test in zip(X_train_list, \
                                                                       X_test_list, \
                                                                       y_train_list, \
                                                                       y_test_list, \
                                                                       date_train_list, \
                                                                       date_test_list\
                                                                      ):
        mae, rmse, r_2, mape, y_pred = run_model_individually(optimal_neurons_layer1, \
                                                           optimal_neurons_layer2, \
                                                           optimal_learning_rate, \
                                                           optimal_batch_size,\
                                                           X_train, \
                                                           X_test, \
                                                           y_train, \
                                                           y_test)
        
        res = []
        res.append(country_)
        res.append(city_)
        res.append(mae)
        res.append(rmse)
        res.append(r_2)
        res.append(mape)
        res.append('fold_' + str(k_fold))
        res.append('PSO+LSTM')

        full_output = pd.DataFrame({
            'actual_value' : y_test,
            'predict_value' : y_pred,
            'Fold_number' : ['fold_' + str(k_fold)] * len(y_test),
            'date' : date_test
        })
        full_output['city'] = city_
        full_output['country'] = country_
        full_output['method'] = 'PSO+LSTM'
        
        summary_value.append(res)
        detail_value.append(full_output)
        
        k_fold = k_fold + 1
        
    summary_df = pd.DataFrame(summary_value,columns=['Country','City','MAE','RMSE','R2','MAPE','K_Fold','method'])
    detail_df = pd.concat(detail_value)
    
    summary_df.to_csv('../_output_PSO+LSTM/result/summary_result_' + country_ + '_' + city_ + '.csv',index=False)
    detail_df.to_csv('../_output_PSO+LSTM/result/details_result_' + country_ + '_' + city_ + '.csv',index=False)
    
    
    

France Marseille


C:\Users\HP\AppData\Local\Temp\ipykernel_19752\2459491258.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_city.drop(columns=['location', 'country', 'latitude', 'longitude', 'generationtime_ms','elevation','Date'],inplace=True)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Iteration 1/20, Best MAE so far: 0.5844728472018536
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Iteration 2/20, Best MAE so far: 0.5454243984374306
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━

C:\Users\HP\AppData\Local\Temp\ipykernel_19752\2459491258.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_city.drop(columns=['location', 'country', 'latitude', 'longitude', 'generationtime_ms','elevation','Date'],inplace=True)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Iteration 1/20, Best MAE so far: 2.369560311136256
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Iteration 2/20, Best MAE so far: 2.1339185104977685
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━

...

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Iteration 1/20, Best MAE so far: 0.6175262702881136
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Iteration 2/20, Best MAE so far: 0.5440899173011817
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━

C:\Users\HP\AppData\Local\Temp\ipykernel_19752\2459491258.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_city.drop(columns=['location', 'country', 'latitude', 'longitude', 'generationtime_ms','elevation','Date'],inplace=True)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Iteration 1/20, Best MAE so far: 0.6117736200152539
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Iteration 2/20, Best MAE so far: 0.6117736200152539
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━

C:\Users\HP\AppData\Local\Temp\ipykernel_19752\2459491258.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_city.drop(columns=['location', 'country', 'latitude', 'longitude', 'generationtime_ms','elevation','Date'],inplace=True)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Iteration 1/20, Best MAE so far: 0.7902167280108181
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Iteration 2/20, Best MAE so far: 0.7902167280108181
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
10/10 ━━━━━━━━━━